# Tier 3: Employment Forecasting - County Level (QCEW)

**Author:** Brandon C. De Los Reyes  
**Affiliation:** Khipu Analytics Research Lab  
**Version:** v1.0  
**Date:** October 8, 2025  
**UUID:** `tier3-employment-counties-qcew-v1`  

---

## Academic Context

This notebook demonstrates **Tier 3: Time Series Analytics** applied to county-level employment data using the BLS Quarterly Census of Employment and Wages (QCEW) program.

### Data Source: QCEW Program

**Important:** County-level employment data comes from a **different BLS program** than state-level data:

- **QCEW (Quarterly Census of Employment and Wages)**
  - Coverage: 3,143 U.S. counties
  - Frequency: Quarterly with **3 monthly employment values per quarter**
  - Method: **CSV file downloads** (not API series IDs)
  - URL Pattern: `http://data.bls.gov/cew/data/api/{year}/{qtr}/area/{area_fips}.csv`
  - Data Available: Employment, wages, establishments, year-over-year changes
  - Industries: All industries and ownership types (42 columns total)

### Key Differences from State Notebook:

1. **Data Access**: Direct CSV downloads vs API POST requests
2. **Frequency**: Quarterly data (with 3 monthly values) vs purely monthly
3. **Identifier**: 5-digit FIPS codes vs 2-digit state codes
4. **Additional Data**: Wages, establishments, year-over-year comparisons included

### Citation

De Los Reyes, B.C. (2025). *Tier 3 Employment Forecasting - County Level (QCEW)*. Khipu Analytics Research Lab. UUID: tier3-employment-counties-qcew-v1.

---

## Learning Objectives

1. Access county-level employment data via BLS QCEW CSV downloads
2. Convert quarterly data with monthly components to continuous time series
3. Apply ARIMA models to county employment patterns
4. Compare employment trends across different geographic scales
5. Generate county-specific employment forecasts

## Prerequisites

- **Tier 1**: Descriptive Analytics (data manipulation fundamentals)
- **Tier 2**: Predictive Analytics (regression and forecasting basics)
- **Tier 3 State Notebook**: State-level employment forecasting (LAUS)

## Next Steps

- **Tier 3 Metro Notebook**: Metropolitan area employment forecasting (Metro LAUS)
- **Tier 4**: Clustering analysis of county employment patterns
- **Tier 5**: Ensemble methods for multi-county forecasting

## Business Applications

- Local economic development planning
- Site selection analysis for business expansion
- Workforce development program targeting
- Regional labor market assessment
- County-level policy impact evaluation

## 1. Configuration and Setup

In [1]:
# Execution tracking and reproducibility import sys import os # Add project root to path for imports project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..')) if project_root not in sys.path: sys.path.insert(0, project_root) try: from src.khipu_analytics.execution_tracking import setup_notebook_tracking metadata = setup_notebook_tracking( notebook_name="Tier3_Employment_Forecasting_Counties_QCEW.ipynb", version="v1.0", seed=42, save_log=True ) print(" Execution tracking initialized") print(f" Session ID: {metadata.get('session_id', 'N/A')}") except Exception as e: print(f"WARNING: Execution tracking not available: {e}") print("Continuing without tracking...") # Set manual seed for reproducibility import random import numpy as np random.seed(42) np.random.seed(42)

⚠️  Execution tracking not available: unexpected indent (execution_tracking.py, line 31)
Continuing without tracking...


In [2]:
# QCEW Configuration QCEW_CONFIG = { # Data source settings "base_url": "http://data.bls.gov/cew/data/api", # Time period "start_year": 2019, "end_year": 2024, # Industry and ownership filters "industry_code": "10", # 10 = All industries (NAICS) "ownership_code": "0", # 0 = Total, all ownerships # Default counties for quick testing "default_counties": [ "51059", # Fairfax County, VA "06037", # Los Angeles County, CA "48201", # Harris County, TX ], # Data validation "min_employment": 100, # Minimum employment to consider valid "max_missing_quarters": 2, # Max allowed missing quarters in time series } print(" QCEW configuration loaded") print(f" Time period: {QCEW_CONFIG['start_year']}-{QCEW_CONFIG['end_year']}") print(f" Industry: All industries (code {QCEW_CONFIG['industry_code']})") print(f" Default test counties: {len(QCEW_CONFIG['default_counties'])}")

✅ QCEW configuration loaded
📅 Time period: 2019-2024
🏢 Industry: All industries (code 10)
📍 Default test counties: 3


## 2. Import Libraries

In [3]:
# Core libraries import pandas as pd import numpy as np from datetime import datetime, timedelta import urllib.request from io import StringIO import warnings warnings.filterwarnings('ignore') # Visualization import plotly.graph_objects as go import plotly.express as px from plotly.subplots import make_subplots # Time series analysis from statsmodels.tsa.seasonal import seasonal_decompose from statsmodels.tsa.stattools import adfuller from statsmodels.tsa.arima.model import ARIMA from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # Forecasting try: from prophet import Prophet PROPHET_AVAILABLE = True except ImportError: PROPHET_AVAILABLE = False print("WARNING: Prophet not available. Install with: pip install prophet") # Interactive widgets (optional) try: import ipywidgets as widgets from IPython.display import display, HTML, clear_output WIDGETS_AVAILABLE = True except ImportError: WIDGETS_AVAILABLE = False print("WARNING: ipywidgets not available. Interactive selection disabled.") print(" Install with: pip install ipywidgets") print(" All required libraries imported successfully") print(f" Plotly version: {px.__version__ if hasattr(px, '__version__') else 'Unknown'}") print(f" Prophet available: {PROPHET_AVAILABLE}") print(f" Interactive widgets available: {WIDGETS_AVAILABLE}")

⚠️  Prophet not available. Install with: pip install prophet
⚠️  ipywidgets not available. Interactive selection disabled.
   Install with: pip install ipywidgets
✅ All required libraries imported successfully
📊 Plotly version: Unknown
🔮 Prophet available: False
🎛️  Interactive widgets available: False


## 3. QCEW Data Access Functions

These functions implement the **official BLS method** for accessing QCEW data via CSV downloads.

In [4]:
def qcewGetAreaData(year, qtr, area): """ Official BLS function to retrieve QCEW data for a specific area and quarter. Parameters: ----------- year : str Year (e.g., '2023') qtr : str Quarter ('1', '2', '3', or '4') area : str 5-digit FIPS code (e.g., '51059' for Fairfax County, VA) Returns: -------- list : 2D array with header row and data rows Example: -------- data = qcewGetAreaData('2023', '1', '51059') # Fairfax County Q1 2023 """ base_url = QCEW_CONFIG['base_url'] url = f"{base_url}/{year}/{qtr}/area/{area}.csv" try: response = urllib.request.urlopen(url) csv_data = response.read().decode('utf-8') response.close() # Parse CSV into 2D array lines = csv_data.strip().split('\n') data = [line.split(',') for line in lines] return data except Exception as e: print(f" Error retrieving data for area {area}, {year} Q{qtr}: {e}") return None def qcewGetAreaDataFrame(year, qtr, area): """ Enhanced version that returns pandas DataFrame instead of raw array. Parameters: ----------- year : str Year (e.g., '2023') qtr : str Quarter ('1', '2', '3', or '4') area : str 5-digit FIPS code Returns: -------- pd.DataFrame : QCEW data with 42 columns including employment, wages, establishments Key Columns: ------------ - area_fips: 5-digit county FIPS code - industry_code: NAICS industry code ('10' = all industries) - own_code: Ownership code ('0' = all ownerships) - year, qtr: Time identifiers - month1_emplvl, month2_emplvl, month3_emplvl: Monthly employment (3 values per quarter!) - avg_wkly_wage: Average weekly wage - qtrly_estabs: Number of establishments - oty_month*_emplvl_chg: Year-over-year employment change """ base_url = QCEW_CONFIG['base_url'] url = f"{base_url}/{year}/{qtr}/area/{area}.csv" try: response = urllib.request.urlopen(url) csv_data = response.read().decode('utf-8') response.close() # Parse as DataFrame df = pd.read_csv(StringIO(csv_data)) # Add metadata columns df['fetch_timestamp'] = datetime.now() return df except Exception as e: print(f" Error retrieving DataFrame for area {area}, {year} Q{qtr}: {e}") return pd.DataFrame() def fetch_qcew_multi_year(area_fips, start_year=None, end_year=None): """ Fetch QCEW data for multiple years and quarters for a specific county. Parameters: ----------- area_fips : str 5-digit county FIPS code start_year : int, optional Starting year (default from config) end_year : int, optional Ending year (default from config) Returns: -------- pd.DataFrame : Combined data for all quarters in time range """ start_year = start_year or QCEW_CONFIG['start_year'] end_year = end_year or QCEW_CONFIG['end_year'] all_data = [] total_quarters = (end_year - start_year + 1) * 4 successful = 0 print(f" Fetching QCEW data for area {area_fips}...") print(f" Period: {start_year} Q1 to {end_year} Q4 ({total_quarters} quarters)") for year in range(start_year, end_year + 1): for qtr in ['1', '2', '3', '4']: df = qcewGetAreaDataFrame(str(year), qtr, area_fips) if not df.empty: all_data.append(df) successful += 1 if all_data: combined = pd.concat(all_data, ignore_index=True) print(f" Successfully retrieved {successful}/{total_quarters} quarters") print(f" Total records: {len(combined):,}") return combined else: print(f" No data retrieved for area {area_fips}") return pd.DataFrame() print(" QCEW data access functions defined") print(" Functions available:") print(" - qcewGetAreaData(): Official BLS function (returns raw array)") print(" - qcewGetAreaDataFrame(): Enhanced pandas version") print(" - fetch_qcew_multi_year(): Multi-year fetching with progress tracking")

✅ QCEW data access functions defined
📋 Functions available:
   - qcewGetAreaData(): Official BLS function (returns raw array)
   - qcewGetAreaDataFrame(): Enhanced pandas version
   - fetch_qcew_multi_year(): Multi-year fetching with progress tracking


## 4. County FIPS Dictionary

Comprehensive mapping of county FIPS codes to names and states. Starting with major counties for testing.

In [5]:
# Major U.S. Counties FIPS Dictionary # Format: {fips: {'state': 'XX', 'county': 'Name', 'full_name': 'Name County, State'}} COUNTY_FIPS = { # Virginia "51059": {"state": "VA", "county": "Fairfax", "full_name": "Fairfax County, Virginia"}, "51013": {"state": "VA", "county": "Arlington", "full_name": "Arlington County, Virginia"}, "51510": {"state": "VA", "county": "Alexandria", "full_name": "Alexandria City, Virginia"}, "51107": {"state": "VA", "county": "Loudoun", "full_name": "Loudoun County, Virginia"}, # California "06037": {"state": "CA", "county": "Los Angeles", "full_name": "Los Angeles County, California"}, "06073": {"state": "CA", "county": "San Diego", "full_name": "San Diego County, California"}, "06059": {"state": "CA", "county": "Orange", "full_name": "Orange County, California"}, "06085": {"state": "CA", "county": "Santa Clara", "full_name": "Santa Clara County, California"}, "06075": {"state": "CA", "county": "San Francisco", "full_name": "San Francisco County, California"}, # Texas "48201": {"state": "TX", "county": "Harris", "full_name": "Harris County, Texas"}, "48113": {"state": "TX", "county": "Dallas", "full_name": "Dallas County, Texas"}, "48029": {"state": "TX", "county": "Bexar", "full_name": "Bexar County, Texas"}, "48453": {"state": "TX", "county": "Travis", "full_name": "Travis County, Texas"}, # Florida "12086": {"state": "FL", "county": "Miami-Dade", "full_name": "Miami-Dade County, Florida"}, "12011": {"state": "FL", "county": "Broward", "full_name": "Broward County, Florida"}, "12103": {"state": "FL", "county": "Pinellas", "full_name": "Pinellas County, Florida"}, "12095": {"state": "FL", "county": "Orange", "full_name": "Orange County, Florida"}, # New York "36061": {"state": "NY", "county": "New York", "full_name": "New York County (Manhattan), New York"}, "36047": {"state": "NY", "county": "Kings", "full_name": "Kings County (Brooklyn), New York"}, "36081": {"state": "NY", "county": "Queens", "full_name": "Queens County, New York"}, "36005": {"state": "NY", "county": "Bronx", "full_name": "Bronx County, New York"}, # Illinois "17031": {"state": "IL", "county": "Cook", "full_name": "Cook County, Illinois"}, "17043": {"state": "IL", "county": "DuPage", "full_name": "DuPage County, Illinois"}, # Pennsylvania "42101": {"state": "PA", "county": "Philadelphia", "full_name": "Philadelphia County, Pennsylvania"}, "42003": {"state": "PA", "county": "Allegheny", "full_name": "Allegheny County, Pennsylvania"}, # Arizona "04013": {"state": "AZ", "county": "Maricopa", "full_name": "Maricopa County, Arizona"}, "04019": {"state": "AZ", "county": "Pima", "full_name": "Pima County, Arizona"}, # Ohio "39035": {"state": "OH", "county": "Cuyahoga", "full_name": "Cuyahoga County, Ohio"}, "39049": {"state": "OH", "county": "Franklin", "full_name": "Franklin County, Ohio"}, # North Carolina "37119": {"state": "NC", "county": "Mecklenburg", "full_name": "Mecklenburg County, North Carolina"}, "37183": {"state": "NC", "county": "Wake", "full_name": "Wake County, North Carolina"}, # Michigan "26163": {"state": "MI", "county": "Wayne", "full_name": "Wayne County, Michigan"}, "26125": {"state": "MI", "county": "Oakland", "full_name": "Oakland County, Michigan"}, # Washington "53033": {"state": "WA", "county": "King", "full_name": "King County, Washington"}, "53061": {"state": "WA", "county": "Snohomish", "full_name": "Snohomish County, Washington"}, # Massachusetts "25025": {"state": "MA", "county": "Suffolk", "full_name": "Suffolk County, Massachusetts"}, "25017": {"state": "MA", "county": "Middlesex", "full_name": "Middlesex County, Massachusetts"}, # Georgia "13121": {"state": "GA", "county": "Fulton", "full_name": "Fulton County, Georgia"}, "13089": {"state": "GA", "county": "DeKalb", "full_name": "DeKalb County, Georgia"}, "13135": {"state": "GA", "county": "Gwinnett", "full_name": "Gwinnett County, Georgia"}, # Colorado "08031": {"state": "CO", "county": "Denver", "full_name": "Denver County, Colorado"}, "08001": {"state": "CO", "county": "Adams", "full_name": "Adams County, Colorado"}, "08005": {"state": "CO", "county": "Arapahoe", "full_name": "Arapahoe County, Colorado"}, # Tennessee "47157": {"state": "TN", "county": "Shelby", "full_name": "Shelby County, Tennessee"}, "47037": {"state": "TN", "county": "Davidson", "full_name": "Davidson County, Tennessee"}, # Maryland "24031": {"state": "MD", "county": "Montgomery", "full_name": "Montgomery County, Maryland"}, "24003": {"state": "MD", "county": "Anne Arundel", "full_name": "Anne Arundel County, Maryland"}, "24005": {"state": "MD", "county": "Baltimore", "full_name": "Baltimore County, Maryland"}, # Wisconsin "55079": {"state": "WI", "county": "Milwaukee", "full_name": "Milwaukee County, Wisconsin"}, "55025": {"state": "WI", "county": "Dane", "full_name": "Dane County, Wisconsin"}, # Minnesota "27053": {"state": "MN", "county": "Hennepin", "full_name": "Hennepin County, Minnesota"}, "27123": {"state": "MN", "county": "Ramsey", "full_name": "Ramsey County, Minnesota"}, # Missouri "29189": {"state": "MO", "county": "St. Louis", "full_name": "St. Louis County, Missouri"}, "29095": {"state": "MO", "county": "Jackson", "full_name": "Jackson County, Missouri"}, # Indiana "18097": {"state": "IN", "county": "Marion", "full_name": "Marion County, Indiana"}, "18089": {"state": "IN", "county": "Lake", "full_name": "Lake County, Indiana"}, # Oregon "41051": {"state": "OR", "county": "Multnomah", "full_name": "Multnomah County, Oregon"}, "41067": {"state": "OR", "county": "Washington", "full_name": "Washington County, Oregon"}, # Nevada "32003": {"state": "NV", "county": "Clark", "full_name": "Clark County, Nevada"}, "32031": {"state": "NV", "county": "Washoe", "full_name": "Washoe County, Nevada"}, # South Carolina "45045": {"state": "SC", "county": "Greenville", "full_name": "Greenville County, South Carolina"}, "45019": {"state": "SC", "county": "Charleston", "full_name": "Charleston County, South Carolina"}, # Louisiana "22071": {"state": "LA", "county": "Orleans", "full_name": "Orleans Parish, Louisiana"}, "22033": {"state": "LA", "county": "East Baton Rouge", "full_name": "East Baton Rouge Parish, Louisiana"}, # Oklahoma "40109": {"state": "OK", "county": "Oklahoma", "full_name": "Oklahoma County, Oklahoma"}, "40143": {"state": "OK", "county": "Tulsa", "full_name": "Tulsa County, Oklahoma"}, # Connecticut "09003": {"state": "CT", "county": "Hartford", "full_name": "Hartford County, Connecticut"}, "09009": {"state": "CT", "county": "New Haven", "full_name": "New Haven County, Connecticut"}, # Kentucky "21111": {"state": "KY", "county": "Jefferson", "full_name": "Jefferson County, Kentucky"}, "21117": {"state": "KY", "county": "Kenton", "full_name": "Kenton County, Kentucky"}, # New Jersey "34003": {"state": "NJ", "county": "Bergen", "full_name": "Bergen County, New Jersey"}, "34013": {"state": "NJ", "county": "Essex", "full_name": "Essex County, New Jersey"}, "34017": {"state": "NJ", "county": "Hudson", "full_name": "Hudson County, New Jersey"}, # New Mexico "35001": {"state": "NM", "county": "Bernalillo", "full_name": "Bernalillo County, New Mexico"}, # Alabama "01073": {"state": "AL", "county": "Jefferson", "full_name": "Jefferson County, Alabama"}, "01089": {"state": "AL", "county": "Madison", "full_name": "Madison County, Alabama"}, # Utah "49035": {"state": "UT", "county": "Salt Lake", "full_name": "Salt Lake County, Utah"}, "49049": {"state": "UT", "county": "Utah", "full_name": "Utah County, Utah"}, # Kansas "20091": {"state": "KS", "county": "Johnson", "full_name": "Johnson County, Kansas"}, "20173": {"state": "KS", "county": "Sedgwick", "full_name": "Sedgwick County, Kansas"}, # Iowa "19153": {"state": "IA", "county": "Polk", "full_name": "Polk County, Iowa"}, "19113": {"state": "IA", "county": "Linn", "full_name": "Linn County, Iowa"}, # Nebraska "31055": {"state": "NE", "county": "Douglas", "full_name": "Douglas County, Nebraska"}, "31109": {"state": "NE", "county": "Lancaster", "full_name": "Lancaster County, Nebraska"}, } # Group counties by state for hierarchical selection COUNTIES_BY_STATE = {} for fips, info in COUNTY_FIPS.items(): state = info['state'] if state not in COUNTIES_BY_STATE: COUNTIES_BY_STATE[state] = [] COUNTIES_BY_STATE[state].append({ 'fips': fips, 'county': info['county'], 'full_name': info['full_name'] }) # Sort counties within each state for state in COUNTIES_BY_STATE: COUNTIES_BY_STATE[state] = sorted(COUNTIES_BY_STATE[state], key=lambda x: x['county']) print(f" County FIPS dictionary loaded") print(f" Total counties available: {len(COUNTY_FIPS)}") print(f" States covered: {len(COUNTIES_BY_STATE)}") print(f"\n Sample counties:") for fips in list(COUNTY_FIPS.keys())[:5]: print(f" {fips}: {COUNTY_FIPS[fips]['full_name']}") print(" ...") print(f"\nINSIGHT: Note: This is a subset of major counties. Full list of 3,143 counties can be added.")

✅ County FIPS dictionary loaded
📍 Total counties available: 84
🗺️  States covered: 35

📋 Sample counties:
   51059: Fairfax County, Virginia
   51013: Arlington County, Virginia
   51510: Alexandria City, Virginia
   51107: Loudoun County, Virginia
   06037: Los Angeles County, California
   ...

💡 Note: This is a subset of major counties. Full list of 3,143 counties can be added.


## 4b. Load Complete County FIPS Dictionary (Optional)

**Option 1:** Use the curated 95-county dictionary above (fast, tested, covers major metros)  
**Option 2:** Load complete 3,143-county dictionary from external file for full coverage

In [ ]:
# Load complete FIPS dictionary (optional - for full 3,143 county coverage) import json from pathlib import Path # Option to load complete dictionary LOAD_COMPLETE_FIPS = False # Set to True to load all 3,143 counties if LOAD_COMPLETE_FIPS: fips_file = Path("county_fips_complete.json") if fips_file.exists(): with open(fips_file, 'r') as f: fips_data = json.load(f) COUNTY_FIPS = fips_data.get('counties', {}) print(f" Loaded complete FIPS dictionary: {len(COUNTY_FIPS):,} counties") print(f" Coverage: All 50 states + DC + territories") else: print(f"WARNING: File not found: {fips_file}") print(f" Using curated 95-county dictionary instead") print(f" To generate: python generate_complete_fips.py") else: print(f" Using curated dictionary: {len(COUNTY_FIPS)} counties") print(f"INSIGHT: Tip: Set LOAD_COMPLETE_FIPS = True for full 3,143 county coverage") # Print statistics print(f"\n Coverage Summary:") print(f" Total counties: {len(COUNTY_FIPS):,}") print(f" States represented: {len(COUNTIES_BY_STATE)}") print(f" Sample counties: {list(COUNTY_FIPS.keys())[:5]}")

## 4c. Interactive County Selection Widget

Use dropdown menus to select county by state, then by county name. This provides an intuitive alternative to manually setting FIPS codes.

In [ ]:
# Interactive county selection using ipywidgets if WIDGETS_AVAILABLE: from ipywidgets import interact, widgets from IPython.display import display, clear_output # Create state selection options state_options = sorted(list(COUNTIES_BY_STATE.keys())) # Global variables for selection selected_state_widget = None selected_county_widget = None selected_fips_widget = None def create_county_selector(): """Create interactive county selector with cascading dropdowns.""" # State dropdown state_dropdown = widgets.Dropdown( options=state_options, value=state_options[0] if state_options else None, description='State:', style={'description_width': 'initial'}, layout=widgets.Layout(width='300px') ) # County dropdown (empty initially) county_dropdown = widgets.Dropdown( options=[], description='County:', style={'description_width': 'initial'}, layout=widgets.Layout(width='400px') ) # FIPS code display fips_label = widgets.Label(value="FIPS Code: Not selected") # Selected info display info_output = widgets.Output() def update_counties(change): """Update county dropdown when state changes.""" selected_state = change['new'] if selected_state in COUNTIES_BY_STATE: counties = COUNTIES_BY_STATE[selected_state] county_dropdown.options = [ (c['full_name'], c['fips']) for c in counties ] if counties: county_dropdown.value = counties[0]['fips'] def update_info(change): """Update info display when county changes.""" selected_fips_code = change['new'] if selected_fips_code in COUNTY_FIPS: county_info = COUNTY_FIPS[selected_fips_code] fips_label.value = f"FIPS Code: {selected_fips_code}" with info_output: clear_output(wait=True) print(f" Selected: {county_info['full_name']}") print(f" FIPS: {selected_fips_code}") print(f" State: {county_info['state']}") print(f"\nINSIGHT: To use this county, run:") print(f" selected_fips = '{selected_fips_code}'") print(f" selected_county = COUNTY_FIPS[selected_fips]['full_name']") # Update global variables global selected_fips_widget, selected_county_widget, selected_state_widget selected_fips_widget = selected_fips_code selected_county_widget = county_info['full_name'] selected_state_widget = county_info['state'] # Connect event handlers state_dropdown.observe(update_counties, names='value') county_dropdown.observe(update_info, names='value') # Initialize first state update_counties({'new': state_dropdown.value}) # Create UI layout ui = widgets.VBox([ widgets.HTML("<h4> Interactive County Selection</h4>"), state_dropdown, county_dropdown, fips_label, info_output, widgets.HTML("<hr>") ]) return ui # Display the selector county_selector_ui = create_county_selector() display(county_selector_ui) print(" Interactive county selector ready") print(" Select a state, then a county from the dropdowns above") else: print("WARNING: ipywidgets not available. Using manual FIPS selection.") print("INSIGHT: Install with: pip install ipywidgets") print(" Manually set: selected_fips = '51059' (example)") # Show available states print(f"\n Available states ({len(COUNTIES_BY_STATE)}):") for state in sorted(COUNTIES_BY_STATE.keys()): county_count = len(COUNTIES_BY_STATE[state]) print(f" {state}: {county_count} counties")

## 4d. Batch Processing Functions

Functions for processing multiple counties simultaneously, comparing employment trends across regions, and generating comparative analysis.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed from tqdm.auto import tqdm def fetch_county_data_parallel(fips_list, start_year=None, end_year=None, max_workers=5): """ Fetch QCEW data for multiple counties in parallel. Parameters: ----------- fips_list : list List of 5-digit county FIPS codes start_year : int, optional Starting year (default from config) end_year : int, optional Ending year (default from config) max_workers : int Maximum parallel download threads (default: 5) Returns: -------- dict : {fips: DataFrame} mapping for successfully downloaded counties """ results = {} failed = [] print(f" Starting parallel download for {len(fips_list)} counties...") print(f" Using {max_workers} parallel workers") def download_county(fips): """Download single county data.""" try: df = fetch_qcew_multi_year(fips, start_year, end_year) if not df.empty: return (fips, df, None) else: return (fips, None, "No data retrieved") except Exception as e: return (fips, None, str(e)) # Execute parallel downloads with progress bar with ThreadPoolExecutor(max_workers=max_workers) as executor: futures = {executor.submit(download_county, fips): fips for fips in fips_list} with tqdm(total=len(fips_list), desc="Downloading county data") as pbar: for future in as_completed(futures): fips, df, error = future.result() pbar.update(1) if df is not None: results[fips] = df else: failed.append((fips, error)) # Print summary print(f"\n Successfully downloaded: {len(results)}/{len(fips_list)} counties") if failed: print(f" Failed downloads: {len(failed)}") for fips, error in failed[:5]: # Show first 5 failures county_name = COUNTY_FIPS.get(fips, {}).get('full_name', fips) print(f" - {county_name}: {error}") if len(failed) > 5: print(f" ... and {len(failed) - 5} more") return results def process_counties_batch(fips_list, analysis_type='descriptive'): """ Process multiple counties and generate comparative analysis. Parameters: ----------- fips_list : list List of county FIPS codes to analyze analysis_type : str Type of analysis: 'descriptive', 'forecast', or 'comparison' Returns: -------- dict : Results for each county """ # Download data for all counties county_data = fetch_county_data_parallel(fips_list) # Process each county results = {} print(f"\n Processing {len(county_data)} counties...") for fips, df_raw in tqdm(county_data.items(), desc="Processing counties"): try: # Convert to monthly time series df_monthly = convert_qcew_to_monthly(df_raw, fips) # Calculate statistics stats = { 'fips': fips, 'county_name': COUNTY_FIPS.get(fips, {}).get('full_name', fips), 'total_records': len(df_monthly), 'avg_employment': df_monthly['employment'].mean(), 'latest_employment': df_monthly['employment'].iloc[-1] if len(df_monthly) > 0 else None, 'employment_growth': ( (df_monthly['employment'].iloc[-1] - df_monthly['employment'].iloc[0]) / df_monthly['employment'].iloc[0] * 100 ) if len(df_monthly) > 1 else None, 'data': df_monthly } results[fips] = stats except Exception as e: print(f"WARNING: Error processing {fips}: {e}") print(f" Processed {len(results)} counties successfully") return results def compare_counties_visualization(county_results, metric='employment'): """ Create comparison visualization for multiple counties. Parameters: ----------- county_results : dict Results dictionary from process_counties_batch() metric : str Metric to compare (default: 'employment') Returns: -------- plotly.graph_objs.Figure : Comparison chart """ import plotly.graph_objects as go from plotly.subplots import make_subplots fig = make_subplots( rows=2, cols=1, subplot_titles=( f'{metric.title()} Time Series Comparison', 'Growth Rates Comparison' ), vertical_spacing=0.15, row_heights=[0.6, 0.4] ) # Time series comparison for fips, data in county_results.items(): df = data['data'] county_name = data['county_name'] fig.add_trace( go.Scatter( x=df['date'], y=df[metric], mode='lines', name=county_name, hovertemplate=f'{county_name}<br>Date: %{{x}}<br>{metric.title()}: %{{y:,.0f}}<extra></extra>' ), row=1, col=1 ) # Growth rates bar chart growth_data = [ (data['county_name'], data.get('employment_growth', 0)) for fips, data in county_results.items() if data.get('employment_growth') is not None ] if growth_data: counties_sorted = sorted(growth_data, key=lambda x: x[1], reverse=True) names, growths = zip(*counties_sorted) colors = ['green' if g > 0 else 'red' for g in growths] fig.add_trace( go.Bar( x=list(names), y=list(growths), marker_color=colors, name='Growth Rate', hovertemplate='%{x}<br>Growth: %{y:.2f}%<extra></extra>', showlegend=False ), row=2, col=1 ) # Update layout fig.update_xaxes(title_text="Date", row=1, col=1) fig.update_yaxes(title_text=metric.title(), row=1, col=1) fig.update_xaxes(title_text="County", row=2, col=1) fig.update_yaxes(title_text="Growth Rate (%)", row=2, col=1) fig.update_layout( height=900, title_text=f"Multi-County {metric.title()} Comparison", hovermode='x unified', showlegend=True, legend=dict( orientation="v", yanchor="top", y=0.95, xanchor="right", x=1.15 ) ) return fig print(" Batch processing functions defined") print(" Available functions:") print(" - fetch_county_data_parallel(): Download multiple counties in parallel") print(" - process_counties_batch(): Process and analyze multiple counties") print(" - compare_counties_visualization(): Create comparison charts") print("\nINSIGHT: Example usage:") print(" fips_list = ['51059', '06037', '48201'] # Fairfax, LA, Harris") print(" results = process_counties_batch(fips_list)") print(" fig = compare_counties_visualization(results)") print(" fig.show()")

## 4e. Multi-County Comparison Example (Optional)

Demonstrate batch processing by comparing employment trends across multiple counties. Uncomment and run to test.

In [ ]:
# Multi-County Comparison Example # Uncomment to run comparison analysis across multiple counties ENABLE_MULTI_COUNTY_COMPARISON = False # Set to True to enable if ENABLE_MULTI_COUNTY_COMPARISON: print(" Running multi-county comparison...") # Select counties to compare (major tech hubs example) comparison_counties = [ "51059", # Fairfax County, VA (DC tech) "06085", # Santa Clara County, CA (Silicon Valley) "53033", # King County, WA (Seattle) "48453", # Travis County, TX (Austin) "13121" # Fulton County, GA (Atlanta) ] print(f"\n Selected counties for comparison:") for fips in comparison_counties: if fips in COUNTY_FIPS: print(f" - {COUNTY_FIPS[fips]['full_name']}") # Process counties in batch print(f"\n⏳ Processing {len(comparison_counties)} counties...") county_results = process_counties_batch(comparison_counties, analysis_type='comparison') # Generate comparison visualization if county_results: print(f"\n Creating comparison visualization...") comparison_fig = compare_counties_visualization(county_results, metric='employment') comparison_fig.show() # Print summary statistics print(f"\n Summary Statistics:") print(f"{'County':<40} {'Avg Employment':>15} {'Growth Rate':>12}") print("-" * 70) for fips, data in sorted( county_results.items(), key=lambda x: x[1].get('employment_growth', 0), reverse=True ): county_name = data['county_name'][:38] avg_emp = data.get('avg_employment', 0) growth = data.get('employment_growth', 0) print(f"{county_name:<40} {avg_emp:>15,.0f} {growth:>11.2f}%") print(f"\n Multi-county comparison complete!") else: print(f" No county data retrieved for comparison") else: print("ℹ Multi-county comparison disabled") print("INSIGHT: Set ENABLE_MULTI_COUNTY_COMPARISON = True to run comparison") print("\n Example comparison counties (tech hubs):") example_counties = [ ("51059", "Fairfax County, VA"), ("06085", "Santa Clara County, CA"), ("53033", "King County, WA"), ("48453", "Travis County, TX"), ("13121", "Fulton County, GA") ] for fips, name in example_counties: print(f" - {fips}: {name}")

## 5. Test Data Retrieval

Quick test to validate the QCEW CSV download method works correctly.

In [6]:
# Test with Fairfax County, VA (51059) test_area = "51059" test_county = COUNTY_FIPS[test_area]['full_name'] print(f"🧪 Testing data retrieval for: {test_county}") print(f" FIPS Code: {test_area}") print("="*60) # Fetch single quarter to verify connection df_test = qcewGetAreaDataFrame('2023', '1', test_area) if not df_test.empty: print(f"\n Successfully retrieved Q1 2023 data") print(f" Records returned: {len(df_test):,}") print(f" Columns: {len(df_test.columns)}") # Filter to total employment df_total = df_test[ (df_test['industry_code'] == QCEW_CONFIG['industry_code']) & (df_test['own_code'] == QCEW_CONFIG['ownership_code']) ] if not df_total.empty: row = df_total.iloc[0] print(f"\n Total Employment (All Industries):") print(f" Month 1 (January): {row['month1_emplvl']:,.0f}") print(f" Month 2 (February): {row['month2_emplvl']:,.0f}") print(f" Month 3 (March): {row['month3_emplvl']:,.0f}") print(f"\n Average Weekly Wage: ${row['avg_wkly_wage']:,.2f}") print(f" Establishments: {row['qtrly_estabs']:,.0f}") print(f"\n Data validation successful!") else: print(f"WARNING: No records found for industry code {QCEW_CONFIG['industry_code']} and ownership {QCEW_CONFIG['ownership_code']}") else: print(f" Failed to retrieve data for {test_county}") print(f" This may indicate an issue with the FIPS code or BLS API availability.")

🧪 Testing data retrieval for: Fairfax County, Virginia
📍 FIPS Code: 51059

✅ Successfully retrieved Q1 2023 data
📊 Records returned: 2,029
📋 Columns: 43
⚠️  No records found for industry code 10 and ownership 0

✅ Successfully retrieved Q1 2023 data
📊 Records returned: 2,029
📋 Columns: 43
⚠️  No records found for industry code 10 and ownership 0


## 6. Fetch Multi-Year County Data

Retrieve complete time series data for the selected county.

In [7]:
# Fetch complete time series for default county selected_fips = QCEW_CONFIG['default_counties'][0] # Fairfax County selected_county = COUNTY_FIPS[selected_fips]['full_name'] print(f" Fetching complete employment time series...") print(f" County: {selected_county}") print(f" Period: {QCEW_CONFIG['start_year']} - {QCEW_CONFIG['end_year']}") print("="*60) # Fetch all quarters df_raw = fetch_qcew_multi_year( selected_fips, QCEW_CONFIG['start_year'], QCEW_CONFIG['end_year'] ) if not df_raw.empty: print(f"\n Raw data summary:") print(f" Total records: {len(df_raw):,}") print(f" Date range: {df_raw['year'].min()} Q{df_raw['qtr'].min()} to {df_raw['year'].max()} Q{df_raw['qtr'].max()}") print(f" Unique industries: {df_raw['industry_code'].nunique()}") print(f" Unique ownership types: {df_raw['own_code'].nunique()}") else: print(f"\n No data retrieved. Cannot proceed with analysis.")

📊 Fetching complete employment time series...
📍 County: Fairfax County, Virginia
📅 Period: 2019 - 2024
📥 Fetching QCEW data for area 51059...
📅 Period: 2019 Q1 to 2024 Q4 (24 quarters)
✅ Successfully retrieved 24/24 quarters
📊 Total records: 48,376

📊 Raw data summary:
   Total records: 48,376
   Date range: 2019 Q1 to 2024 Q4
   Unique industries: 1991
   Unique ownership types: 5
✅ Successfully retrieved 24/24 quarters
📊 Total records: 48,376

📊 Raw data summary:
   Total records: 48,376
   Date range: 2019 Q1 to 2024 Q4
   Unique industries: 1991
   Unique ownership types: 5


## 7. Data Processing: Quarterly to Monthly Conversion

Convert quarterly QCEW data (which contains 3 monthly values per quarter) into a continuous monthly time series.

In [8]:
def convert_qcew_to_monthly(df_qcew, area_fips): """ Convert quarterly QCEW data to monthly time series. Each quarter contains 3 monthly employment values: - Q1: month1 (Jan), month2 (Feb), month3 (Mar) - Q2: month1 (Apr), month2 (May), month3 (Jun) - Q3: month1 (Jul), month2 (Aug), month3 (Sep) - Q4: month1 (Oct), month2 (Nov), month3 (Dec) Parameters: ----------- df_qcew : pd.DataFrame Raw QCEW data with quarterly structure area_fips : str County FIPS code for filtering Returns: -------- pd.DataFrame : Monthly time series with columns: - date: Monthly datetime - employment: Employment level - avg_wkly_wage: Average weekly wage - establishments: Number of establishments """ # Filter to total employment (all industries, all ownerships) df_total = df_qcew[ (df_qcew['industry_code'] == QCEW_CONFIG['industry_code']) & (df_qcew['own_code'] == QCEW_CONFIG['ownership_code']) ].copy() if df_total.empty: print(f"WARNING: No total employment data found for area {area_fips}") return pd.DataFrame() # Sort by year and quarter df_total = df_total.sort_values(['year', 'qtr']).reset_index(drop=True) # Build monthly records monthly_records = [] # Quarter to month mapping qtr_to_months = { '1': [1, 2, 3], # Q1: Jan, Feb, Mar '2': [4, 5, 6], # Q2: Apr, May, Jun '3': [7, 8, 9], # Q3: Jul, Aug, Sep '4': [10, 11, 12] # Q4: Oct, Nov, Dec } for idx, row in df_total.iterrows(): year = int(row['year']) qtr = str(row['qtr']) months = qtr_to_months.get(qtr, []) # Extract 3 monthly values month_values = [ row['month1_emplvl'], row['month2_emplvl'], row['month3_emplvl'] ] # Create records for each month for i, month_num in enumerate(months): if pd.notna(month_values[i]) and month_values[i] > 0: date = pd.Timestamp(year=year, month=month_num, day=1) monthly_records.append({ 'date': date, 'employment': month_values[i], 'avg_wkly_wage': row['avg_wkly_wage'], 'establishments': row['qtrly_estabs'], 'year': year, 'month': month_num, 'quarter': qtr }) # Create DataFrame df_monthly = pd.DataFrame(monthly_records) if not df_monthly.empty: df_monthly = df_monthly.sort_values('date').reset_index(drop=True) df_monthly['county_name'] = COUNTY_FIPS[area_fips]['full_name'] df_monthly['fips'] = area_fips return df_monthly # Convert to monthly time series print(f" Converting quarterly data to monthly time series...") df_employment = convert_qcew_to_monthly(df_raw, selected_fips) if not df_employment.empty: print(f"\n Monthly time series created") print(f" Date range: {df_employment['date'].min().strftime('%Y-%m')} to {df_employment['date'].max().strftime('%Y-%m')}") print(f" Total months: {len(df_employment)}") print(f"\n Employment statistics:") print(f" Mean: {df_employment['employment'].mean():,.0f}") print(f" Min: {df_employment['employment'].min():,.0f} ({df_employment[df_employment['employment'] == df_employment['employment'].min()]['date'].iloc[0].strftime('%Y-%m')})") print(f" Max: {df_employment['employment'].max():,.0f} ({df_employment[df_employment['employment'] == df_employment['employment'].max()]['date'].iloc[0].strftime('%Y-%m')})") print(f"\n Wage statistics:") print(f" Mean weekly wage: ${df_employment['avg_wkly_wage'].mean():,.2f}") print(f" Range: ${df_employment['avg_wkly_wage'].min():,.2f} - ${df_employment['avg_wkly_wage'].max():,.2f}") # Display first few records print(f"\n Sample data (first 5 months):") display(df_employment[['date', 'employment', 'avg_wkly_wage', 'establishments']].head()) else: print(f"\n Failed to create monthly time series")

🔄 Converting quarterly data to monthly time series...
⚠️  No total employment data found for area 51059

❌ Failed to create monthly time series


## 8. Exploratory Visualization

Visualize the employment time series and wage trends.

In [9]:
# Create multi-panel visualization fig = make_subplots( rows=3, cols=1, subplot_titles=( f'Employment Trend - {selected_county}', 'Average Weekly Wage', 'Number of Establishments' ), vertical_spacing=0.1, row_heights=[0.4, 0.3, 0.3] ) # Employment trend fig.add_trace( go.Scatter( x=df_employment['date'], y=df_employment['employment'], mode='lines+markers', name='Employment', line=dict(color='#2E86AB', width=2), marker=dict(size=4) ), row=1, col=1 ) # Wage trend fig.add_trace( go.Scatter( x=df_employment['date'], y=df_employment['avg_wkly_wage'], mode='lines+markers', name='Avg Weekly Wage', line=dict(color='#A23B72', width=2), marker=dict(size=4) ), row=2, col=1 ) # Establishments trend fig.add_trace( go.Scatter( x=df_employment['date'], y=df_employment['establishments'], mode='lines+markers', name='Establishments', line=dict(color='#F18F01', width=2), marker=dict(size=4) ), row=3, col=1 ) # Update layout fig.update_xaxes(title_text="Date", row=3, col=1) fig.update_yaxes(title_text="Employment", row=1, col=1) fig.update_yaxes(title_text="Dollars", row=2, col=1) fig.update_yaxes(title_text="Count", row=3, col=1) fig.update_layout( height=900, showlegend=False, title_text=f"QCEW Employment Analytics - {selected_county}", title_x=0.5, hovermode='x unified' ) fig.show() print(f"\n Key observations:") emp_change = df_employment['employment'].iloc[-1] - df_employment['employment'].iloc[0] emp_pct_change = (emp_change / df_employment['employment'].iloc[0]) * 100 print(f" Employment change: {emp_change:+,.0f} ({emp_pct_change:+.1f}%)") wage_change = df_employment['avg_wkly_wage'].iloc[-1] - df_employment['avg_wkly_wage'].iloc[0] wage_pct_change = (wage_change / df_employment['avg_wkly_wage'].iloc[0]) * 100 print(f" Wage change: ${wage_change:+,.2f} ({wage_pct_change:+.1f}%)") est_change = df_employment['establishments'].iloc[-1] - df_employment['establishments'].iloc[0] est_pct_change = (est_change / df_employment['establishments'].iloc[0]) * 100 print(f" Establishment change: {est_change:+,.0f} ({est_pct_change:+.1f}%)")

KeyError: 'date'

## 9. Time Series Stationarity Testing

Before applying ARIMA, we test if the employment series is stationary using the Augmented Dickey-Fuller (ADF) test.

In [ ]:
def test_stationarity(timeseries, variable_name='Employment'): """ Perform Augmented Dickey-Fuller test for stationarity. Parameters: ----------- timeseries : pd.Series Time series data to test variable_name : str Name of variable for display Returns: -------- dict : Test results with p-value and stationarity status """ print(f" Testing stationarity for {variable_name}") print("="*60) # Perform ADF test result = adfuller(timeseries.dropna(), autolag='AIC') output = { 'test_statistic': result[0], 'p_value': result[1], 'lags_used': result[2], 'observations': result[3], 'critical_values': result[4], 'is_stationary': result[1] < 0.05 } print(f"ADF Test Statistic: {output['test_statistic']:.4f}") print(f"P-value: {output['p_value']:.4f}") print(f"Lags used: {output['lags_used']}") print(f"Number of observations: {output['observations']}") print("\nCritical Values:") for key, value in output['critical_values'].items(): print(f" {key}: {value:.4f}") if output['is_stationary']: print(f"\n Result: Series is STATIONARY (p-value < 0.05)") print(f" → Can proceed with ARIMA modeling") else: print(f"\nWARNING: Result: Series is NON-STATIONARY (p-value >= 0.05)") print(f" → Need to difference the series or use seasonal decomposition") return output # Test employment series stationarity_result = test_stationarity(df_employment['employment'], 'Employment') # If non-stationary, test first difference if not stationarity_result['is_stationary']: print("\n" + "="*60) print(" Testing first difference...") df_employment['employment_diff'] = df_employment['employment'].diff() stationarity_diff = test_stationarity( df_employment['employment_diff'].dropna(), 'Employment (First Difference)' )

## 10. Seasonal Decomposition

Decompose the employment series into trend, seasonal, and residual components.

In [ ]:
# Perform seasonal decomposition print(f" Performing seasonal decomposition...") print(f" Period: 12 months (annual seasonality)") # Set employment as index for decomposition df_decomp = df_employment.set_index('date')[['employment']].copy() # Decompose with 12-month period decomposition = seasonal_decompose( df_decomp['employment'], model='additive', period=12 ) # Create visualization fig = make_subplots( rows=4, cols=1, subplot_titles=( 'Original Series', 'Trend Component', 'Seasonal Component', 'Residual Component' ), vertical_spacing=0.08, row_heights=[0.25, 0.25, 0.25, 0.25] ) # Original fig.add_trace( go.Scatter( x=df_decomp.index, y=df_decomp['employment'], mode='lines', name='Original', line=dict(color='#2E86AB', width=1.5) ), row=1, col=1 ) # Trend fig.add_trace( go.Scatter( x=df_decomp.index, y=decomposition.trend, mode='lines', name='Trend', line=dict(color='#A23B72', width=2) ), row=2, col=1 ) # Seasonal fig.add_trace( go.Scatter( x=df_decomp.index, y=decomposition.seasonal, mode='lines', name='Seasonal', line=dict(color='#F18F01', width=1.5) ), row=3, col=1 ) # Residual fig.add_trace( go.Scatter( x=df_decomp.index, y=decomposition.resid, mode='lines', name='Residual', line=dict(color='#6A994E', width=1) ), row=4, col=1 ) fig.update_xaxes(title_text="Date", row=4, col=1) fig.update_yaxes(title_text="Employment", row=1, col=1) fig.update_yaxes(title_text="Trend", row=2, col=1) fig.update_yaxes(title_text="Seasonal", row=3, col=1) fig.update_yaxes(title_text="Residual", row=4, col=1) fig.update_layout( height=1000, showlegend=False, title_text=f"Seasonal Decomposition - {selected_county}", title_x=0.5 ) fig.show() # Print decomposition insights print(f"\n Decomposition insights:") print(f" Trend range: {decomposition.trend.min():,.0f} to {decomposition.trend.max():,.0f}") print(f" Seasonal amplitude: {decomposition.seasonal.max() - decomposition.seasonal.min():,.0f}") print(f" Residual std dev: {decomposition.resid.std():,.0f}") # Identify strongest seasonal months seasonal_by_month = pd.DataFrame({ 'month': df_decomp.index.month, 'seasonal': decomposition.seasonal }).groupby('month')['seasonal'].mean().sort_values(ascending=False) print(f"\n Strongest seasonal months (employment above trend):") for month, value in seasonal_by_month.head(3).items(): month_name = pd.Timestamp(2024, month, 1).strftime('%B') print(f" {month_name}: +{value:,.0f}")

## 11. ARIMA Model Selection and Training

Determine optimal ARIMA parameters and train the model.

In [ ]:
# ARIMA Configuration ARIMA_CONFIG = { 'order': (1, 1, 1), # (p, d, q) - AR, differencing, MA 'seasonal_order': (1, 1, 1, 12), # (P, D, Q, s) - seasonal components 'forecast_periods': 12 # 12 months ahead } print(f" ARIMA Model Configuration") print("="*60) print(f"Order (p,d,q): {ARIMA_CONFIG['order']}") print(f"Seasonal Order (P,D,Q,s): {ARIMA_CONFIG['seasonal_order']}") print(f"Forecast horizon: {ARIMA_CONFIG['forecast_periods']} months") # Prepare training data train_data = df_employment['employment'].values train_dates = df_employment['date'].values print(f"\n Training data:") print(f" Observations: {len(train_data)}") print(f" Date range: {df_employment['date'].min().strftime('%Y-%m')} to {df_employment['date'].max().strftime('%Y-%m')}") # Train ARIMA model print(f"\n Training ARIMA model...") try: model = ARIMA( train_data, order=ARIMA_CONFIG['order'], seasonal_order=ARIMA_CONFIG['seasonal_order'] ) arima_fit = model.fit() print(f" Model trained successfully") print(f"\n Model Summary:") print(f" AIC: {arima_fit.aic:.2f}") print(f" BIC: {arima_fit.bic:.2f}") print(f" Log Likelihood: {arima_fit.llf:.2f}") # Display model parameters print(f"\n Model Parameters:") for name, value in arima_fit.params.items(): print(f" {name}: {value:.4f}") except Exception as e: print(f" Error training ARIMA model: {e}") print(f" Try adjusting model parameters or checking data quality") arima_fit = None

## 12. Generate Employment Forecast

Create 12-month ahead forecast with confidence intervals.

In [ ]:
if arima_fit is not None: # Generate forecast print(f" Generating {ARIMA_CONFIG['forecast_periods']}-month forecast...") forecast_result = arima_fit.forecast(steps=ARIMA_CONFIG['forecast_periods']) forecast_values = forecast_result # Get confidence intervals forecast_df = arima_fit.get_forecast(steps=ARIMA_CONFIG['forecast_periods']) forecast_ci = forecast_df.conf_int() # Create forecast dates last_date = df_employment['date'].max() forecast_dates = pd.date_range( start=last_date + pd.DateOffset(months=1), periods=ARIMA_CONFIG['forecast_periods'], freq='MS' ) # Build forecast DataFrame df_forecast = pd.DataFrame({ 'date': forecast_dates, 'forecast': forecast_values, 'lower_ci': forecast_ci.iloc[:, 0], 'upper_ci': forecast_ci.iloc[:, 1] }) print(f" Forecast generated successfully") print(f"\n Forecast summary:") print(f" Period: {df_forecast['date'].min().strftime('%Y-%m')} to {df_forecast['date'].max().strftime('%Y-%m')}") print(f" Mean forecast: {df_forecast['forecast'].mean():,.0f}") print(f" Range: {df_forecast['forecast'].min():,.0f} to {df_forecast['forecast'].max():,.0f}") # Calculate forecast trend forecast_change = df_forecast['forecast'].iloc[-1] - df_forecast['forecast'].iloc[0] forecast_pct = (forecast_change / df_forecast['forecast'].iloc[0]) * 100 print(f" 12-month change: {forecast_change:+,.0f} ({forecast_pct:+.2f}%)") # Display forecast table print(f"\n Monthly Forecast:") display(df_forecast[['date', 'forecast', 'lower_ci', 'upper_ci']].round(0)) # Visualization fig = go.Figure() # Historical data fig.add_trace(go.Scatter( x=df_employment['date'], y=df_employment['employment'], mode='lines', name='Historical', line=dict(color='#2E86AB', width=2) )) # Forecast fig.add_trace(go.Scatter( x=df_forecast['date'], y=df_forecast['forecast'], mode='lines+markers', name='Forecast', line=dict(color='#A23B72', width=2, dash='dash'), marker=dict(size=6) )) # Confidence interval fig.add_trace(go.Scatter( x=pd.concat([df_forecast['date'], df_forecast['date'][::-1]]), y=pd.concat([df_forecast['upper_ci'], df_forecast['lower_ci'][::-1]]), fill='toself', fillcolor='rgba(162, 59, 114, 0.2)', line=dict(color='rgba(255,255,255,0)'), name='95% Confidence Interval', showlegend=True )) fig.update_layout( title=f"Employment Forecast - {selected_county}", xaxis_title="Date", yaxis_title="Employment", hovermode='x unified', height=600 ) fig.show() else: print("WARNING: Cannot generate forecast - ARIMA model training failed")

## 13. Model Diagnostics

Evaluate model performance and residuals.

In [ ]:
if arima_fit is not None: print(f" Performing model diagnostics...") # Get residuals residuals = arima_fit.resid # Calculate performance metrics from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score fitted_values = arima_fit.fittedvalues actual_values = train_data # Align fitted values with actual (fitted may be shorter due to differencing) min_len = min(len(fitted_values), len(actual_values)) fitted_aligned = fitted_values[-min_len:] actual_aligned = actual_values[-min_len:] mae = mean_absolute_error(actual_aligned, fitted_aligned) rmse = np.sqrt(mean_squared_error(actual_aligned, fitted_aligned)) mape = np.mean(np.abs((actual_aligned - fitted_aligned) / actual_aligned)) * 100 print(f"\n Model Performance Metrics:") print(f" MAE (Mean Absolute Error): {mae:,.0f}") print(f" RMSE (Root Mean Squared Error): {rmse:,.0f}") print(f" MAPE (Mean Absolute Percentage Error): {mape:.2f}%") # Residual analysis print(f"\n Residual Analysis:") print(f" Mean: {residuals.mean():.2f}") print(f" Std Dev: {residuals.std():,.2f}") print(f" Min: {residuals.min():,.0f}") print(f" Max: {residuals.max():,.0f}") # Create diagnostic plots fig = make_subplots( rows=2, cols=2, subplot_titles=( 'Residuals Over Time', 'Residual Histogram', 'Actual vs Fitted Values', 'Residual Q-Q Plot' ) ) # Residuals over time fig.add_trace( go.Scatter( y=residuals, mode='lines', name='Residuals', line=dict(color='#2E86AB', width=1) ), row=1, col=1 ) fig.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=1) # Histogram fig.add_trace( go.Histogram( x=residuals, name='Distribution', marker_color='#A23B72', nbinsx=30 ), row=1, col=2 ) # Actual vs Fitted fig.add_trace( go.Scatter( x=actual_aligned, y=fitted_aligned, mode='markers', name='Actual vs Fitted', marker=dict(color='#F18F01', size=4) ), row=2, col=1 ) # Add diagonal line min_val = min(actual_aligned.min(), fitted_aligned.min()) max_val = max(actual_aligned.max(), fitted_aligned.max()) fig.add_trace( go.Scatter( x=[min_val, max_val], y=[min_val, max_val], mode='lines', name='Perfect Fit', line=dict(color='red', dash='dash'), showlegend=False ), row=2, col=1 ) # Q-Q plot from scipy import stats qq = stats.probplot(residuals, dist="norm") fig.add_trace( go.Scatter( x=qq[0][0], y=qq[0][1], mode='markers', name='Q-Q Plot', marker=dict(color='#6A994E', size=4) ), row=2, col=2 ) # Add reference line fig.add_trace( go.Scatter( x=[qq[0][0].min(), qq[0][0].max()], y=[qq[0][1].min(), qq[0][1].max()], mode='lines', line=dict(color='red', dash='dash'), showlegend=False ), row=2, col=2 ) fig.update_xaxes(title_text="Time", row=1, col=1) fig.update_xaxes(title_text="Residual Value", row=1, col=2) fig.update_xaxes(title_text="Actual Employment", row=2, col=1) fig.update_xaxes(title_text="Theoretical Quantiles", row=2, col=2) fig.update_yaxes(title_text="Residual", row=1, col=1) fig.update_yaxes(title_text="Frequency", row=1, col=2) fig.update_yaxes(title_text="Fitted Employment", row=2, col=1) fig.update_yaxes(title_text="Sample Quantiles", row=2, col=2) fig.update_layout( height=800, showlegend=False, title_text=f"Model Diagnostics - {selected_county}" ) fig.show() print(f"\n Diagnostics complete") else: print("WARNING: Cannot perform diagnostics - ARIMA model not available")

## 14. Prophet Forecasting (Optional)

Alternative forecasting method using Facebook's Prophet library.

In [ ]:
if PROPHET_AVAILABLE: print(f" Training Prophet model...") # Prepare data for Prophet (requires 'ds' and 'y' columns) df_prophet = pd.DataFrame({ 'ds': df_employment['date'], 'y': df_employment['employment'] }) # Initialize and fit Prophet model prophet_model = Prophet( yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False, changepoint_prior_scale=0.05 ) prophet_model.fit(df_prophet) # Create future dataframe future = prophet_model.make_future_dataframe(periods=12, freq='MS') # Generate forecast prophet_forecast = prophet_model.predict(future) print(f" Prophet model trained and forecast generated") # Extract forecast for future periods only df_prophet_forecast = prophet_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12) df_prophet_forecast.columns = ['date', 'forecast', 'lower_ci', 'upper_ci'] print(f"\n Prophet Forecast Summary:") print(f" Mean forecast: {df_prophet_forecast['forecast'].mean():,.0f}") print(f" Range: {df_prophet_forecast['forecast'].min():,.0f} to {df_prophet_forecast['forecast'].max():,.0f}") # Visualization comparing ARIMA and Prophet fig = go.Figure() # Historical fig.add_trace(go.Scatter( x=df_employment['date'], y=df_employment['employment'], mode='lines', name='Historical', line=dict(color='#2E86AB', width=2) )) # ARIMA forecast if arima_fit is not None: fig.add_trace(go.Scatter( x=df_forecast['date'], y=df_forecast['forecast'], mode='lines+markers', name='ARIMA Forecast', line=dict(color='#A23B72', width=2, dash='dash') )) # Prophet forecast fig.add_trace(go.Scatter( x=df_prophet_forecast['date'], y=df_prophet_forecast['forecast'], mode='lines+markers', name='Prophet Forecast', line=dict(color='#F18F01', width=2, dash='dot'), marker=dict(size=6, symbol='diamond') )) fig.update_layout( title=f"Forecast Comparison: ARIMA vs Prophet - {selected_county}", xaxis_title="Date", yaxis_title="Employment", hovermode='x unified', height=600 ) fig.show() else: print("WARNING: Prophet not available. Install with: pip install prophet") print(" Skipping Prophet forecasting...")

## 15. Export Results

Save forecast results and analysis summary.

In [ ]:
import json # Create export directory if it doesn't exist export_dir = os.path.join(project_root, 'reports', 'employment_forecasts') os.makedirs(export_dir, exist_ok=True) # Prepare export data export_data = { 'metadata': { 'county_fips': selected_fips, 'county_name': selected_county, 'analysis_date': datetime.now().strftime('%Y-%m-%d'), 'notebook_version': 'v1.0', 'data_source': 'BLS QCEW', 'time_period': f"{QCEW_CONFIG['start_year']}-{QCEW_CONFIG['end_year']}" }, 'historical_data': { 'observations': len(df_employment), 'date_range': { 'start': df_employment['date'].min().strftime('%Y-%m'), 'end': df_employment['date'].max().strftime('%Y-%m') }, 'employment': { 'mean': float(df_employment['employment'].mean()), 'min': float(df_employment['employment'].min()), 'max': float(df_employment['employment'].max()), 'std': float(df_employment['employment'].std()) }, 'wage': { 'mean': float(df_employment['avg_wkly_wage'].mean()), 'min': float(df_employment['avg_wkly_wage'].min()), 'max': float(df_employment['avg_wkly_wage'].max()) } } } # Add ARIMA forecast if available if arima_fit is not None and 'df_forecast' in locals(): export_data['arima_forecast'] = { 'model_config': ARIMA_CONFIG, 'performance': { 'mae': float(mae), 'rmse': float(rmse), 'mape': float(mape), 'aic': float(arima_fit.aic), 'bic': float(arima_fit.bic) }, 'forecast': df_forecast[['date', 'forecast', 'lower_ci', 'upper_ci']].to_dict('records') } # Add Prophet forecast if available if PROPHET_AVAILABLE and 'df_prophet_forecast' in locals(): export_data['prophet_forecast'] = { 'forecast': df_prophet_forecast.to_dict('records') } # Export to JSON json_filename = f"employment_forecast_{selected_fips}_{datetime.now().strftime('%Y%m%d')}.json" json_path = os.path.join(export_dir, json_filename) with open(json_path, 'w') as f: json.dump(export_data, f, indent=2, default=str) print(f" Results exported successfully") print(f" Location: {json_path}") # Also export forecast as CSV if 'df_forecast' in locals(): csv_filename = f"employment_forecast_{selected_fips}_{datetime.now().strftime('%Y%m%d')}.csv" csv_path = os.path.join(export_dir, csv_filename) df_forecast.to_csv(csv_path, index=False) print(f" CSV exported: {csv_path}") # Export full time series with forecasts df_full = pd.concat([ df_employment[['date', 'employment']].rename(columns={'employment': 'actual'}), df_forecast[['date', 'forecast']].rename(columns={'forecast': 'predicted'}) ], ignore_index=True) full_csv = f"employment_timeseries_{selected_fips}_{datetime.now().strftime('%Y%m%d')}.csv" full_csv_path = os.path.join(export_dir, full_csv) df_full.to_csv(full_csv_path, index=False) print(f" Full time series: {full_csv_path}") print(f"\n Total files exported: 3") print(f" 1. JSON summary with metadata and forecasts") print(f" 2. CSV forecast data") print(f" 3. CSV full time series (historical + forecast)")